# SEQP Personalized Signal Generator
This is a Jupyter notebook designed to create a personalized version of the SEQP test signal. 

1) Type in your callsign and grid square below.
2) Click "Run All Cells."

In [1]:
call = 'KD8OXT'
maidenhead = 'EN91em'

### 0) Housekeeping: Import packages and set audio parameters

In [2]:
# fs = 48000;                   # samples per second
# dt = 1/fs;                    # seconds per sample
# wpm = 20

In [3]:
import morse_talk as mtalk
import pandas as pd
import numpy as np

## 1) Generate pseudorandom signal from user's callsign and grid square.
Notion for a pseudorandom code: generate two seconds of static - one from the callsign, one from the grid square. We'll stick them together. -- Going to leave this out for the moment because python's random number seeds aren't very predictable.

In [4]:
# import random
# hash(call)

In [5]:
def pseudorandom_from_string(s):
    rng = default_rng(list(s.encode('utf-8')))
    return rng.standard_normal(12) # TODO change this to match size you want

The results of this function should be repeatable no matter how many times you run this function, or restart the notebook kernel.

In [6]:
pseudorandom_from_string(call)

array([-1.16492545, -0.20373905, -1.04660338, -0.91713151,  2.1189248 ,
       -0.59934971,  0.09140789, -1.12785716,  1.12018562,  0.81818285,
       -1.23872975,  0.08551055])

In [7]:
pseudorandom_from_string(call)

array([-1.16492545, -0.20373905, -1.04660338, -0.91713151,  2.1189248 ,
       -0.59934971,  0.09140789, -1.12785716,  1.12018562,  0.81818285,
       -1.23872975,  0.08551055])

In [8]:
call_static = pseudorandom_from_string(call)
grid_static = pseudorandom_from_string(maidenhead)

In [10]:
import matplotlib.pyplot as plt

## 2) Generate Morse signal from user's callsign. 
This signal is at 20 WPM. All callsigns must fit in the same time block... and should be received by RBN.

In [8]:
msg = 'TEST TEST TEST de ' + call + " " + call + " " + call + "  " + maidenhead+ "  " + maidenhead
# msg = msg * 3
print(msg)

TEST TEST TEST de KD8OXT KD8OXT KD8OXT  EN91em  EN91em


Let's convert the signal to an on-off signal:

In [53]:
msg_cw = mtalk.encode(msg, encoding_type='binary')
msg_cw = np.array(list(msg_cw))
msg_cw = pd.DataFrame(msg_cw)

We have to make sure this fits into a standard time frame, so let's prallocate an array of desired length, convert it to a list, and then add it in:

## 3) Concatenate CW and radar chirps

Now we add in the lightsaber noises. Er, chirps. 

Chirps from [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.7659302.svg)](https://doi.org/10.5281/zenodo.7659302).

In [62]:
print(msg_cw.shape)
chirps.shape

(477, 1)


(515999, 1)

In [42]:
data = pd.read_csv('https://zenodo.org/record/7659302/files/seqp-test.csv?download=1')
chirps = data.to_numpy()#.transpose()
chirps

array([[0.        ],
       [0.        ],
       [0.        ],
       ...,
       [0.70699947],
       [0.70699989],
       [0.70699999]])

In [60]:
signal = np.concatenate((msg_cw, chirps))
signal

array([['1'],
       ['1'],
       ['1'],
       ...,
       [0.706999467635388],
       [0.706999894841548],
       [0.706999993427597]], dtype=object)

## 4) Save File
Let's save the resulting .CSV and .WAV files with mnemonic filenames that include callsign and grid square.

In [61]:
filename = 'SEQP Test Signal ' + call + ' ' + maidenhead
df = pd.DataFrame(signal)
df.to_csv(filename+'.csv')